# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

## 5.5. Modeling
Ở đây nhóm sử dụng 2 model là Decision Tree và Logistic Regression cùng với GridSearchCV để xác thực các siêu tham số. Khi sử dụng GridSearchCV cũng mặc định sử dụng phương pháp k-folds cross validation với k = 5

### 5.5.1. Decision Tree
Tập siêu tham số xác thực:
- max_depth: Độ sâu tối đa của cây = [5, 10, 15, 20]
- min_sample_split: Số sample tối thiểu để một node tiếp tục chia thành node con = [2, 5, 10]
- min_sample_leaf: Số sample tối thiếu của một lá = [1, 2, 4]
- criterion: Thước đo chất lượng phân chia ở mỗi node = [gini, entropy]

GridSearchCV với các tham số trên sẽ tiến hành tối ưu hóa các siêu tham số thay vì phải tự chạy nhiều vòng for thủ công. Tiêu chí tìm kiếm của GridSearchCV ở đây là accuracy trên tập test càng cao càng tốt

In [ ]:
def decision_tree(X, Y):
    param_grid = {'max_depth': [int(x) for x in np.linspace(5, 20, num = 4)],
                   'min_samples_split': [2, 5, 10],
                   'min_samples_leaf': [1, 2, 4],
                   'criterion': ['gini', 'entropy']
                  }
    dt = DecisionTreeClassifier()
    dt_grid = GridSearchCV(estimator = dt, param_grid = param_grid, scoring = 'accuracy')
    dt_grid.fit(X, Y)
    return dt_grid

#### 5.5.1.1. Tập sample 1


In [ ]:
tree_grid_res1 = decision_tree(X1, Y1)

In [ ]:
tree_grid_res1.best_params_

In [ ]:
tree_grid_res1.best_score_

Với tập sample1, độ chính xác tối đa trên tập test là 75.5% với các tham số:
- criterion là entropy
- max_depth là 5
- min_samples_leaf là 1
- min_samples_split là 5

#### 5.5.1.2. Tập sample 2

In [ ]:
tree_grid_res2 = decision_tree(X2, Y2)

In [ ]:
tree_grid_res2.best_params_

In [ ]:
tree_grid_res2.best_score_

Với tập sample2, độ chính xác tối đa trên tập test là 77.6% với các tham số:
- criterion là entropy
- max_depth là 5
- min_samples_leaf là 1
- min_samples_split là 2